In [33]:
from utils.utils import load_captions_txt

captions_dict = load_captions_txt("../data/Flickr8k_text/captions.txt")

# Poi estrai la lista flat delle caption per il tokenizer
all_captions = []
for caps in captions_dict.values():
    all_captions.extend(caps)

from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(all_captions)

vocab_size = len(tokenizer.word_index) + 1

def max_caption_length(captions):
    return max(len(c.split()) for c in captions)

max_length = max_caption_length(all_captions)

print(f"Vocabulary size: {vocab_size}")
print(f"Max caption length: {max_length}")


Vocabulary size: 8497
Max caption length: 40


In [34]:
import pickle
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from tensorflow.keras.optimizers import Adam

# Carica feature estratte (encoder output)
with open("features/image_features.pkl", "rb") as f:
    image_features = pickle.load(f)

# Input feature immagine
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# Input sequenze testo (caption)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

# Combina
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

# Modello finale
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model.summary()


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_23      │ (None, 40)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_22      │ (None, 2048)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_11        │ (None, 40, 256)   │  2,175,232 │ input_layer_23[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_22          │ (None, 2048)      │          0 │ input_layer_22[0… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_23          │ (None, 40, 256)   │          0 │ embedding_11[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_11        │ (None, 40)        │          0 │ input_layer_23[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_33 (Dense)    │ (None, 256)       │    524,544 │ dropout_22[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_11 (LSTM)      │ (None, 256)       │    525,312 │ dropout_23[0][0], │
│                     │                   │            │ not_equal_11[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_11 (Add)        │ (None, 256)       │          0 │ dense_33[0][0],   │
│                     │                   │            │ lstm_11[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_34 (Dense)    │ (None, 256)       │     65,792 │ add_11[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_35 (Dense)    │ (None, 8497)      │  2,183,729 │ dense_34[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,474,609 (20.88 MB)

 Trainable params: 5,474,609 (20.88 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
import tensorflow as tf
from utils.utils import data_generator

batch_size = 64

output_signature = (
    (
        tf.TensorSpec(shape=(None, 2048), dtype=tf.float32),    # Feature immagini
        tf.TensorSpec(shape=(None, max_length), dtype=tf.int32) # Sequenze input
    ),
    tf.TensorSpec(shape=(None, vocab_size), dtype=tf.float32)   # Target one-hot
)

# Crea dataset da generatore
dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(captions_dict, image_features, tokenizer, max_length, vocab_size, batch_size),
    output_signature=output_signature
)

steps = len(captions_dict) // batch_size

# Debug: controlla un batch dal dataset
for batch in dataset.take(1):
    (X1, X2), y = batch
    print("X1 shape:", X1.shape)
    print("X2 shape:", X2.shape)
    print("y shape:", y.shape)


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was ((tf.float32, tf.int32), tf.float32), but the yielded element was ([array([[0.6061626 , 0.02493388, 0.77150846, ..., 0.15593892, 1.7264771 ,
        0.09147231],
       [0.6061626 , 0.02493388, 0.77150846, ..., 0.15593892, 1.7264771 ,
        0.09147231],
       [0.6061626 , 0.02493388, 0.77150846, ..., 0.15593892, 1.7264771 ,
        0.09147231],
       ...,
       [0.22027051, 1.0440986 , 0.22833312, ..., 0.9854808 , 0.43434474,
        0.57423586],
       [0.22027051, 1.0440986 , 0.22833312, ..., 0.9854808 , 0.43434474,
        0.57423586],
       [0.22027051, 1.0440986 , 0.22833312, ..., 0.9854808 , 0.43434474,
        0.57423586]], dtype=float32), array([[  0,   0,   0, ...,   0,   0,   3],
       [  0,   0,   0, ...,   0,   3,   2],
     

In [ ]:
model.fit(
    dataset,
    epochs=20,
    steps_per_epoch=steps,
    verbose=1
)

# Salva modello
model.save("model_caption.h5")

# Salva tokenizer in json
tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w') as f:
    f.write(tokenizer_json)
